In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
import sqlalchemy as sq
import re
from tqdm import tqdm
import jieba

In [2]:
defaultList = []
cedictList = []

In [3]:
# defaultdict = r"""C:\Users\jw\Anaconda3\Lib\site-packages\jieba\dict.txt.big.txt"""
defaultdict = r"""C:\Users\jjw4418\AppData\Local\Continuum\anaconda3\Lib\site-packages\jieba\dict.txt"""

In [4]:
defaultdict

'C:\\Users\\jjw4418\\AppData\\Local\\Continuum\\anaconda3\\Lib\\site-packages\\jieba\\dict.txt'

In [5]:
jieba.set_dictionary(defaultdict)
jieba.load_userdict("cedict.txt")

Building prefix dict from C:\Users\jjw4418\AppData\Local\Continuum\anaconda3\Lib\site-packages\jieba\dict.txt ...
Loading model from cache C:\Users\jjw4418\AppData\Local\Temp\jieba.ucc289b289abee56ea5308f1a6d99cc4e.cache
Loading model cost 0.818 seconds.
Prefix dict has been built succesfully.


In [6]:
jieba.set_dictionary(defaultdict)
jieba.load_userdict("cedict.txt")

Building prefix dict from C:\Users\jjw4418\AppData\Local\Continuum\anaconda3\Lib\site-packages\jieba\dict.txt ...
Loading model from cache C:\Users\jjw4418\AppData\Local\Temp\jieba.ucc289b289abee56ea5308f1a6d99cc4e.cache
Loading model cost 0.846 seconds.
Prefix dict has been built succesfully.


In [7]:
data = pd.read_json("core_documents.json")

In [8]:
data.head()

,doc_country,doc_page_name,doc_url,doc_date,doc_content
0,尼日利亚,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,1393891200000,2014年3月3日，驻拉各斯总领事刘侃拜会拉各斯州州长法萧拉。\n\n 刘总领事感谢法萧拉...
1,瑞士,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,1305590400000,2011年5月16日，驻瑞士大使吴恳在大使馆会见到馆的中国侨联代表团和参加座谈的旅瑞华侨华人...
10,阿联酋,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,1360713600000,2013年2月7日，驻阿联酋大使黄杰民出席由阿联酋扎耶德大学孔子学院和英国布莱顿学校阿布扎比...
100,乌克兰,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,1325635200000,2012年1月4日，中乌两国迎来建交20周年纪念日。当天，驻乌克兰大使张喜云在使馆举办中乌建...
1000,奥地利,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,1307664000000,2011年6月8日，驻奥地利大使史明德出席在维也纳大学举行的“2011中国教育展”开幕式。中...


In [9]:
data['doc_date'] = pd.to_datetime(data["doc_date"], unit='ms', origin='unix')

In [10]:
# data.reset_index(level=0, inplace=True)

In [11]:
data.head()

,doc_country,doc_page_name,doc_url,doc_date,doc_content
0,尼日利亚,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2014-03-04,2014年3月3日，驻拉各斯总领事刘侃拜会拉各斯州州长法萧拉。\n\n 刘总领事感谢法萧拉...
1,瑞士,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-05-17,2011年5月16日，驻瑞士大使吴恳在大使馆会见到馆的中国侨联代表团和参加座谈的旅瑞华侨华人...
10,阿联酋,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2013-02-13,2013年2月7日，驻阿联酋大使黄杰民出席由阿联酋扎耶德大学孔子学院和英国布莱顿学校阿布扎比...
100,乌克兰,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2012-01-04,2012年1月4日，中乌两国迎来建交20周年纪念日。当天，驻乌克兰大使张喜云在使馆举办中乌建...
1000,奥地利,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-06-10,2011年6月8日，驻奥地利大使史明德出席在维也纳大学举行的“2011中国教育展”开幕式。中...


In [17]:
data['doc_date'] = data['doc_date'].astype('str').copy()

In [18]:
data.head()

,doc_country,doc_page_name,doc_url,doc_date,doc_content
0,尼日利亚,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2014-03-04,2014年3月3日，驻拉各斯总领事刘侃拜会拉各斯州州长法萧拉。\n\n 刘总领事感谢法萧拉...
1,瑞士,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-05-17,2011年5月16日，驻瑞士大使吴恳在大使馆会见到馆的中国侨联代表团和参加座谈的旅瑞华侨华人...
10,阿联酋,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2013-02-13,2013年2月7日，驻阿联酋大使黄杰民出席由阿联酋扎耶德大学孔子学院和英国布莱顿学校阿布扎比...
100,乌克兰,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2012-01-04,2012年1月4日，中乌两国迎来建交20周年纪念日。当天，驻乌克兰大使张喜云在使馆举办中乌建...
1000,奥地利,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-06-10,2011年6月8日，驻奥地利大使史明德出席在维也纳大学举行的“2011中国教育展”开幕式。中...


In [14]:
data['doc_date'] = data['doc_date'].dt.date.copy()

In [19]:
data.head()

,doc_country,doc_page_name,doc_url,doc_date,doc_content
0,尼日利亚,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2014-03-04,2014年3月3日，驻拉各斯总领事刘侃拜会拉各斯州州长法萧拉。\n\n 刘总领事感谢法萧拉...
1,瑞士,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-05-17,2011年5月16日，驻瑞士大使吴恳在大使馆会见到馆的中国侨联代表团和参加座谈的旅瑞华侨华人...
10,阿联酋,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2013-02-13,2013年2月7日，驻阿联酋大使黄杰民出席由阿联酋扎耶德大学孔子学院和英国布莱顿学校阿布扎比...
100,乌克兰,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2012-01-04,2012年1月4日，中乌两国迎来建交20周年纪念日。当天，驻乌克兰大使张喜云在使馆举办中乌建...
1000,奥地利,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-06-10,2011年6月8日，驻奥地利大使史明德出席在维也纳大学举行的“2011中国教育展”开幕式。中...


In [20]:
data.dtypes

doc_country      object
doc_page_name    object
doc_url          object
doc_date         object
doc_content      object
dtype: object

In [21]:
data.to_json("coreDocumentsLines.json", orient="records", lines=True)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.to_csv("coreDocuments2.csv", encoding='utf-8')

In [ ]:
data.head()

In [ ]:
data.to_csv("coreDocuments2.csv", index_label="index"encoding='utf-8')

In [ ]:
data['doc_content'] = data['doc_content'].astype("str").copy()

In [ ]:
data.dtypes

In [ ]:
data.head()

In [ ]:
data.to_csv("coreDocuments.txt", index=True, encoding='utf-8')

In [ ]:
# data.to_excel("coreDocuments.xlsx", index=True)

In [ ]:
data['doc_content'] = data['doc_content'].str.replace(pat="\W", repl=" ").copy()

In [ ]:
data.head()

In [ ]:
def my_tokenizer(s):
    return jieba.lcut(s, cut_all=False)

count = CountVectorizer(tokenizer=my_tokenizer, min_df=2)

In [ ]:
countdata = count.fit_transform(data['doc_content'])

In [ ]:
countdata

In [ ]:
countDF = pd.DataFrame(countdata.todense(), index=data.index, columns=count.get_feature_names)

In [ ]:
s = data.loc[4,'doc_content']

In [ ]:
seg_list = jieba.cut(s, cut_all=True)
print("Full Mode: " + " ".join(seg_list))  # 全模式
print()
seg_list = jieba.cut(s, cut_all=False)
print("Default Mode: " + " ".join(seg_list))  # 默认模式

In [ ]:
jieba.lcut(s, cut_all=True)

In [ ]:
len(count.get_feature_names())